In [1]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup as bs
import requests


In [62]:
## filling the table with BeautifulSoup


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source, 'lxml')
tabla = soup.find('table')


tablaome = []

for cositos in tabla.find_all('tr'):


    for barrios in cositos.find_all('td'):

        asd=barrios.text
        tablaome.append(asd)

df = pd.DataFrame(np.array(tablaome).reshape(289,3), columns=['Postcode', 'Borough', 'Neighbourhood'])


    
df['Neighbourhood']=df['Neighbourhood'].str.rstrip('\n')
print(df.head(10))
df.shape

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned


(289, 3)

In [63]:
#Taking out the NA data
dfSinRepetidos = df[df.Borough != 'Not assigned']
dfSinRepetidos.shape

(212, 3)

In [66]:
#Now I'll combine the borough repeated rows
dfDos=dfSinRepetidos.groupby(['Postcode','Borough']).agg({ 'Neighbourhood':lambda x:list(x)}).reset_index()
dfDos.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [34]:
#Clean the squared parenthesis 
prueba=dfDos.copy()
prueba['Neighbourhood']= prueba['Neighbourhood'].astype(str).str.replace('\[|\]|\'', '')
#for limpios in prueba['Neighbourhood']:
#    for limpi in limpios:
#        limpios=limpi.replace('\[|\]|\'', '')
#    print(limpios)
prueba.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [67]:

asd=len(prueba.index)
for n in range(0, asd):
     if prueba.iloc[n,:]['Neighbourhood']=='Not assigned':
        prueba.iloc[n,:]['Neighbourhood']==prueba.iloc[n,:]['Borough']
        


In [68]:
#checking if everything is fine so far
prueba['Neighbourhood'].value_counts()

Northwest                                                                                                                                  1
Adelaide, King, Richmond                                                                                                                   1
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara                                 1
Humber Bay Shores, Mimico South, New Toronto                                                                                               1
Cedarbrae                                                                                                                                  1
Agincourt                                                                                                                                  1
Parkdale, Roncesvalles                                                                                                                     1
East Toronto 

In [72]:
finalDf=prueba.copy()
finalDf.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [74]:
#(I cant seem to make big titles, so I added this line so it takes more space and looks like a bigger deal)
## THIS IS PART 2:
#Downloading the data into a DataFrame:

url = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
coordenadas = pd.read_csv(url)
coordenadas.head(10)
    

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [77]:
#Now joining together both dataframes

union = pd.concat([finalDf[:], coordenadas],axis=1)
union.head(10)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


In [86]:
#theres was repeated column, so I deleted it

union.drop([3])
union.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
